In [ ]:
!pip -q install -U pandas numpy scikit-learn imbalanced-learn

In [ ]:
#import the libraries
import os, json, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Import the dataset
data = pd.read_csv('Placement_Data.csv')

In [ ]:
print(data)

#Remove duplicate


In [ ]:
# Find duplicated rows based on 'name'
duplicates = df[df.duplicated(subset=['name'], keep='first')]

print("Dropped places:")
print(duplicates)

# Then drop duplicates
# data = data.drop_duplicates(subset=['name'])
# data

#Xử lí NAN

In [ ]:
# Đếm số lượng NaN trong cột
print(df['rating'].isnull().sum())

# Check missing ratio
missing_ratio = df.isna().mean()
print(missing_ratio[missing_ratio > 0])

# Drop rows with missing label (mandatory)
df = df.dropna(subset=["label"])

# Fill missing feature values (simple & safe)
for col in df.columns:
    if col == "label":
        continue
    if df[col].dtype == "object":
        df[col] = df[col].fillna("unknown")
    else:
        df[col] = df[col].fillna(df[col].median())

print("After handling missing values:", df.shape)

#Handle missing data SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer
imputer_y = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_y.fit(y.reshape(-1,1))
y = imputer_y.transform(y.reshape(-1,1))

print(y)

#Encoding categorical data that doesn't have binary result

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

print(x)

#Encoding categorical data that have binary result

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 6] = le.fit_transform(x[:, 6])

print(x)

#Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

print(x)

#Handle class imbalance Imblearn

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# One-hot encode categorical features
X_train_enc = pd.get_dummies(X_train, dummy_na=True)
X_val_enc = pd.get_dummies(X_val, dummy_na=True)

# Align columns
X_val_enc = X_val_enc.reindex(columns=X_train_enc.columns, fill_value=0)

# Apply Random Over Sampling (recommended for competition)
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train_enc, y_train)

print("Before ROS:", y_train.value_counts())
print("After ROS:", pd.Series(y_train_bal).value_counts())

#Splitting the data into the Training set and Test set


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["label", "label_encoded"])
y = df["label_encoded"]

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)

#Save to new CSV file

In [ ]:
train_processed = X_train_bal.copy()
train_processed["label"] = y_train_bal

val_processed = X_val_enc.copy()
val_processed["label"] = y_val.values

train_processed.to_csv("train_processed.csv", index=False)
val_processed.to_csv("val_processed.csv", index=False)

print("Saved: train_processed.csv, val_processed.csv")